<a href="https://colab.research.google.com/github/Jaumzinho109/Impactus_Python_Scripts/blob/main/Inflation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CPI e Afins

In [ ]:
import pandas as pd
from fredapi import Fred
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import matplotlib.ticker as mtick

font_path = r"C:\Users\Joao\Downloads\Montserrat\static\Montserrat-Regular.ttf"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
fm.fontManager.addfont(font_path)

fred = Fred(api_key="672d5598c8a41df9397cc5eb92c02d5e")

pc = fred.get_series("SUUR0000SA0L1E")
core_cpi = pd.DataFrame()
core_cpi["Nível de preços do núcleo"] = pd.DataFrame(pc)
core_cpi["Pct Change"] = core_cpi['Nível de preços do núcleo'].pct_change()
core_cpi.index = pd.to_datetime(core_cpi.index)

valores_2024 = core_cpi_24.groupby(core_cpi_24.index.month)["Pct Change"].first()
valores_2025 = core_cpi_25.groupby(core_cpi_25.index.month)["Pct Change"].first()

core_cpi = core_cpi[(core_cpi.index.year >= 2010) & (core_cpi.index.year <= 2019)]


meses = {}
for mes in range(1, 13):
    meses[mes] = core_cpi[core_cpi.index.month == mes]

percentil_10 = pd.DataFrame()
percentil_90 = pd.DataFrame()

for mes in range(1, 13):
    dados_mes = core_cpi[core_cpi.index.month == mes]["Pct Change"]
    p10 = dados_mes.quantile(0.10)
    p90 = dados_mes.quantile(0.90)
    percentil_10.loc[mes, "Percentil 10"] = p10
    percentil_90.loc[mes, "Percentil 90"] = p90

valores_2024 = []
valores_2025 = []

for mes in range(1, 13):
    valor_24 = core_cpi_24[core_cpi_24.index.month == mes]["Pct Change"].values
    valor_25 = core_cpi_25[core_cpi_25.index.month == mes]["Pct Change"].values
    valores_2024.append(valor_24[0] if len(valor_24) > 0 else None)
    valores_2025.append(valor_25[0] if len(valor_25) > 0 else None)

valores_dos_graficos = pd.DataFrame()
valores_dos_graficos["Percentil 10"] = percentil_10["Percentil 10"]
valores_dos_graficos["Percentil 90"] = percentil_90["Percentil 90"]
valores_dos_graficos["Ano de 2024"] = valores_2024
valores_dos_graficos["Ano de 2025"] = valores_2025
valores_dos_graficos["Mediana"] = core_cpi.groupby(core_cpi.index.month)["Pct Change"].median()
valores_dos_graficos.index = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

display(valores_dos_graficos)


In [ ]:
plt.figure(figsize=(18, 10.8))

plt.fill_between(valores_dos_graficos.index, valores_dos_graficos["Percentil 10"], valores_dos_graficos["Percentil 90"], color="grey", alpha=0.3, label="10th-90th (2010-2019)")
plt.plot(valores_dos_graficos.index, valores_dos_graficos["Mediana"], linestyle="dotted", linewidth=2, color="#166083", label="Median")
plt.plot(valores_dos_graficos.index, valores_dos_graficos["Ano de 2024"], marker="o", linewidth=2, color="#082631", label="2024")
plt.plot(valores_dos_graficos.index, valores_dos_graficos["Ano de 2025"], marker="o", linewidth=2, color="#37A6D9", label="2025")
plt.suptitle("Core CPI", fontsize=20, fontweight='bold', fontproperties=font_prop)
plt.legend(frameon=False, fontsize=14, prop=font_prop, loc="upper right")

plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["left"].set_color("#d9d9d9")
plt.gca().spines["bottom"].set_color("#d9d9d9")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))

plt.xlabel("Fonte: FRED | Impactus UFRJ", fontsize=14, labelpad=15, fontproperties=font_prop)
plt.tight_layout()
plt.show()


PCE SA - Main

In [ ]:
#Este é o core PCE
psa = fred.get_series("PCEPILFE")
core_pce_sa = pd.DataFrame()
core_pce_sa["Pct Change"] = pd.DataFrame(psa).pct_change()
core_pce_sa["Pct Change from a year ago"] = pd.DataFrame(psa).pct_change(periods=12)

#Vou pegar limites 90 e 10
psa = fred.get_series("PCEPILFE")
core_pce_sa = pd.DataFrame()
core_pce_sa["Pct Change"] = pd.DataFrame(psa).pct_change()
core_pce_sa["Pct Change from a year ago"] = pd.DataFrame(psa).pct_change(periods=12)

#Vou pegar limites 90 e 10, valores de 2024 e 2025 e mediana para o pct change
pce_graph_values = core_pce_sa[(core_pce_sa.index.year >= 2010) & (core_pce_sa.index.year <= 2019)]
percentil_10_pctchg = pce_graph_values.groupby(pce_graph_values.index.month)["Pct Change"].quantile(0.10)
percentil_90_pctchg = pce_graph_values.groupby(pce_graph_values.index.month)["Pct Change"].quantile(0.90)
mediana_pctchg = pce_graph_values.groupby(pce_graph_values.index.month)["Pct Change"].median()
pce_pctchg_2024 = core_pce_sa[core_pce_sa.index.year == 2024].groupby(core_pce_sa[core_pce_sa.index.year == 2024].index.month)["Pct Change"].first()
pce_pctchg_2025 = core_pce_sa[core_pce_sa.index.year == 2025].groupby(core_pce_sa[core_pce_sa.index.year == 2025].index.month)["Pct Change"].first()

pce_pctchg = pd.DataFrame({
    "Percentil 10": percentil_10_pctchg,
    "Percentil 90": percentil_90_pctchg,
    "Ano de 2024": pce_pctchg_2024,
    "Ano de 2025": pce_pctchg_2025,
    "Mediana": mediana_pctchg
})

pce_pctchg.index = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [ ]:
#Agora vou criar o mesmo para pct change from a year ago
pce_graph_values_ya = core_pce_sa[(core_pce_sa.index.year >= 2009)]
mma3 = pce_graph_values_ya["Pct Change from a year ago"].rolling(window=3).mean()
mma12 = pce_graph_values_ya["Pct Change from a year ago"].rolling(window=12).mean()
mma6 = pce_graph_values_ya["Pct Change from a year ago"].rolling(window=6).mean()
mean_10_19 = core_pce_sa[(core_pce_sa.index.year >= 2010) & (core_pce_sa.index.year <= 2019)]["Pct Change from a year ago"].mean()

pce_graph_values_ya["MMA3"] = mma3
pce_graph_values_ya["MMA6"] = mma6
pce_graph_values_ya["MMA12"] = mma12
pce_graph_values_ya["Mean 2010-2019"] = mean_10_19

pce_ya = pd.DataFrame({
    "MMA3": mma3,
    "MMA6": mma6,
    "MMA12": mma12,
    "Mean 2010-2019": mean_10_19
})

pce_ya.dropna(inplace=True)
pce_ya = pce_ya.drop(pce_ya.index[0])
pce_ya

In [ ]:
plt.figure(figsize=(18, 10.8))
plt.plot(pce_pctchg.index, pce_pctchg["Mediana"], linewidth=2, color="#166083", label="Median")
plt.plot(pce_pctchg.index, pce_pctchg["Ano de 2024"], marker="o", linewidth=2, color="#082631", label="2024")
plt.plot(pce_pctchg.index, pce_pctchg["Ano de 2025"], marker="o", linewidth=2, color="#37A6D9", label="2025")
plt.plot(pce_pctchg.index, pce_pctchg["Percentil 10"], color="grey", ls= "-.")
plt.plot(pce_pctchg.index, pce_pctchg["Percentil 90"], color="grey", ls= "-.")

plt.suptitle("Core CPI", fontsize=20, fontweight='bold', fontproperties=font_prop)
plt.text(0.505, 0.94, "Pct Change MoM %", fontsize=14, fontproperties=font_prop, ha='center', transform=plt.gcf().transFigure)
plt.legend(frameon=False, fontsize=14, prop=font_prop, loc="upper right")

plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["left"].set_color("#d9d9d9")
plt.gca().spines["bottom"].set_color("#d9d9d9")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))

plt.xlabel("Fonte: FRED | Impactus UFRJ", fontsize=14, labelpad=15, fontproperties=font_prop)
plt.tight_layout()
plt.show()


In [ ]:
# Cria a figura
plt.figure(figsize=(18, 10.8))

# Plota as linhas
plt.plot(pce_ya.index, pce_ya["MMA3"], linewidth=2, color="#AFABAB", label="3 MMA", ls= ":")
plt.plot(pce_ya.index, pce_ya["MMA6"], linewidth=2, color="#37A6D9", label="6 MMA", ls= "--")
plt.plot(pce_ya.index, pce_ya["MMA12"], linewidth=2, color="#082631", label="12 MMA")
plt.plot(pce_ya.index, pce_ya["Mean 2010-2019"], linewidth=2, color="#166083", label="Mean 2010-2019")

plt.suptitle("Core PCE", fontsize=20, fontweight='bold', fontproperties=font_prop)
plt.text(0.505, 0.94, "Pct Change YoY %", fontsize=14, fontproperties=font_prop, ha='center', transform=plt.gcf().transFigure)
plt.legend(frameon=False, fontsize=14, prop=font_prop, loc="upper right")

plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["left"].set_color("#c0c0c0")
plt.gca().spines["bottom"].set_color("#c0c0c0")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))

plt.xlabel("Fonte: FRED | Impactus UFRJ", fontsize=14, labelpad=15, fontproperties=font_prop)
plt.ylim(0, 0.07)

plt.text(pce_ya.index[-1], pce_ya["MMA3"].iloc[-1], f'{pce_ya["MMA3"].iloc[-1]:.2%}', color="#AFABAB", fontsize=12, ha='left')
plt.text(pce_ya.index[-1], pce_ya["MMA6"].iloc[-1], f'{pce_ya["MMA6"].iloc[-1]:.2%}', color="#37A6D9", fontsize=12, ha='left')
plt.text(pce_ya.index[-1], pce_ya["MMA12"].iloc[-1], f'{pce_ya["MMA12"].iloc[-1]:.2%}', color="#082631", fontsize=12, ha='left')
plt.text(pce_ya.index[-1], pce_ya["Mean 2010-2019"].iloc[-1], f'{pce_ya["Mean 2010-2019"].iloc[-1]:.2%}', color="#166083", fontsize=12, ha='left')

plt.tight_layout()
plt.show()